<a href="https://colab.research.google.com/github/seulmi0827/fininsight/blob/main/JACE/refine_sentence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q kss flashtext
!pip install -q pandarallel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 44.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.9/757.9 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 18.2 MB/s eta 0:00

In [ ]:
!apt-get update
!apt-get install g++ openjdk-8-jdk -y
!pip install konlpy
!pip install mecab-python
!apt-get install curl -y
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [ ]:
from typing import List
import pandas as pd
import re
from kss import split_sentences
from flashtext import KeywordProcessor
from pandarallel import pandarallel
from konlpy.tag import Mecab

In [ ]:
def expand_aspect_terms(aspect_terms: List[str]) -> List[str]:
    expanded = set()

    for term in aspect_terms:
        if '_' in term:
            clean_term = term.replace('_', ' ')
            expanded.add(clean_term) # 언더바를 띄어쓰기로 대체한 단어 추가
            expanded.add(clean_term.replace(" ", ""))  # 띄어쓰기를 제거한 단어 추가
        elif ' ' in term:
            expanded.add(term) # 띄어쓰기 된 원본 단어 추가
            clean_term = term.replace(' ', '')
            expanded.add(clean_term) # 띄어쓰기 제거된 단어 추가
        else:
            clean_term = term # 언더바도 아니고 띄어쓰기도 아닌 원래 단어
            expanded.add(clean_term)

    return list(expanded)


def extract_aspect_sentences(df, aspect_terms):
    pandarallel.initialize(progress_bar=True, verbose=1)

    expanded_terms = expand_aspect_terms(aspect_terms)

    keyword_processor = KeywordProcessor()
    for term in expanded_terms:
        keyword_processor.add_keyword(term)

    # 정규식 패턴 생성
    pattern_dict = {
        term: re.compile(rf'\b{re.escape(term)}\b') for term in expanded_terms
    }

    def process_content(content):
        results = []
        try:
            # KSS로 문장 분리
            sentences = split_sentences(content)

            for sentence in sentences:
                # 조사 제거 처리
                mecab = Mecab()
                processed_sentence = sentence # 원본문장 복제

                # 문장 형태소 분석
                pos_tagged = mecab.pos(sentence)

                # 뒤에서부터 조사 제거 (위치 변경 방지)
                current_pos = len(sentence)
                for i in range(len(pos_tagged)-1, -1, -1):
                    word, pos = pos_tagged[i]
                    if pos.startswith("J"): # "J", "V", "E", "M", "X", "SL", "SW", "SP", "SF", "SS"
                        word_pos = sentence.rfind(word, 0, current_pos)
                        if word_pos != -1:
                            processed_sentence = processed_sentence[:word_pos] + processed_sentence[word_pos+len(word):]
                            current_pos = word_pos

                # 키워드 검색
                if keyword_processor.extract_keywords(processed_sentence):
                    for term, pattern in pattern_dict.items():
                        if pattern.search(processed_sentence):
                            results.append({'original_sentence': sentence, 'processed_sentence': processed_sentence, 'term': term})
        except Exception as e:
            print(f"오류 발생: {e}")
            pass
        return results

    # 병렬 처리
    all_results = df['content'].parallel_apply(process_content)

    # 결과 합치기
    flattened_results = [item for sublist in all_results for item in sublist]
    return pd.DataFrame(flattened_results)

In [ ]:
# 1. Aspect terms 불러오기
aspect_df = pd.read_csv('/content/drive/MyDrive/fin/ABSA/사회_both_1.csv')
aspect_df_add = pd.read_csv('/content/drive/MyDrive/fin/ABSA/사회_disagreement_gpt_공유용.csv')
aspect_terms1 = aspect_df['단어'].dropna().astype(str).tolist()
aspect_terms2 = aspect_df_add['단어'].dropna().astype(str).tolist()
aspect_terms = aspect_terms1 + aspect_terms2
print(f"로드된 aspect terms: {len(aspect_terms)}개")
print()

# 2. 기사 데이터 불러오기
df = pd.read_csv('/content/drive/MyDrive/fin/전남소멸_10000.csv', encoding='utf-8')
df["content"] = df["content"].fillna("").astype(str)
print(df['content'])
print(f"로드된 기사: {len(df)}개")
print()

# 3. 병렬 처리로 문장 추출 실행
print("문장 추출 시작...")
result_df = extract_aspect_sentences(df, aspect_terms)
result_df

# 4. 결과 저장
result_df.to_csv('/content/drive/MyDrive/fin/0420_사회_전남소멸_aspect_sentences.csv', index=False, encoding='utf-8')
print(f"추출된 문장: {len(result_df)}개")

# 5. 결과 샘플 출력
print("\n결과 샘플:")
print(result_df.head())

로드된 aspect terms: 1929개

0       전남도교육청이 2024년부터 지역 초등학생들에게 전국에서 처음으로 매월 교육수당을 ...
1       전남도교육청이 2024년부터 지역 초등학생들에게 전국에서 처음으로 매월 교육수당을 ...
2       경남 합천군 묘산면 두무산 일대가 새 양수발전소 건설 최종 후보지로 선정됐다. 합천...
3       지자체들도 지방소멸을 막기 위해 새로운 시도를 하고 있습니다. 한계에 부딪힌 양적 ...
4       경남도는 '합천군 두무산 일대'가 정부가 추진 중인 전력 수급 안정화를 위한 신규 ...
                              ...                        
8982    청년 어촌 정착 부담↓ '청년어선임대 시범사업' 실시 '귀어인의 집' 시범사업 추진...
8983    경남도청 진주 이전이 소멸위기 서부경남 발전 도모 우주항공 남강프로젝트 등 산업 문...
8984    지금이 인구절벽 극복할 '마지막 골든타임' 지난 2017년 9월 크리스틴 라가르드 ...
8985    존경하는 도민 여러분! 희망찬 2022년 임인년(壬寅年) 새해가 밝았습니다.  도민...
8986    2022 신년사 존경하는 도민 여러분! 희망찬 2022년 임인년(壬寅年) 새해가 밝...
Name: content, Length: 8987, dtype: object
로드된 기사: 8987개

문장 추출 시작...


추출된 문장: 137321개

결과 샘플:
                                   original_sentence  \
0                   학생수당은 학원과 마트, 음식점 등에서는 사용할 수 없다.   
1  저출생 등으로 지역소멸 위기에 처한 지역에 사는 학생들을 지원하기 위해 교육청이 학...   
2  전남교육청은 도내 22개 시군 가운데 지역소멸 위험지역으로 분류된 16개 군 지역 ...   
3  전남 학생교육수당은 전남교육청 홈페이지를 통해 신청하고, 농협 체크카드를 따로 발급...   
4          전남교육청은 학생수당을 전남 소재 사업체에서만 사용하도록 제한할 방침이다.   

                                  processed_sentence  term  
0                        학생수당 학원 마트, 음식점 등 사용할 수 없다.   음식점  
1  저출생 등 지역소멸 위기 처한 지역 사는 학생들 지원하기 위해 교육청 학생수당 주는...   교육청  
2  전남교육청 내 22개 시군 가운데 지역소멸 위험지역 분류된 16개 군 지역 초등학교...  초등학교  
3  전남 학생교육수당 전남교육청 홈페이지 통해 신청하고, 농협 체크카드 따로 발급받아야...  홈페이지  
4               전남교육청 학생수당 전남 소재 사업체 사용하도록 제한할 방침이다.   사업체  


In [ ]:
# term별 문장 수 계산
term_counts = result_df.groupby('term')['target_sentence'].nunique().reset_index()
term_counts.columns = ['term', 'sentence_count']

# 문장 수 기준 내림차순 정렬
term_counts = term_counts.sort_values('sentence_count', ascending=False)

# 결과 저장
term_counts.to_csv('/content/drive/MyDrive/fin/term_sentence_counts.csv', index=False, encoding='utf-8')

# 상위 10개 term 출력
print("상위 10개 term:")
print(term_counts.head())


상위 10개 term:
       term  sentence_count
236     대통령            1772
767     지난달            1163
157    국민의힘            1073
255     동해안            1025
242  더불어민주당             874


In [ ]:
# 1. Aspect terms 불러오기
aspect_df = pd.read_csv('/content/drive/MyDrive/fin/ABSA/사회_both_1.csv')
aspect_df_add = pd.read_csv('/content/drive/MyDrive/fin/ABSA/사회_disagreement_gpt_공유용.csv')
aspect_terms1 = aspect_df['단어'].dropna().astype(str).tolist()
aspect_terms2 = aspect_df_add['단어'].dropna().astype(str).tolist()
aspect_terms = aspect_terms1 + aspect_terms2
print(f"로드된 aspect terms: {len(aspect_terms)}개")
print()

# 2. 기사 데이터 불러오기
df = pd.read_csv('/content/drive/MyDrive/fin/경북산불.csv', encoding='utf-8')
df["content"] = df["content"].fillna("").astype(str)
print(df['content'])
print(f"로드된 기사: {len(df)}개")
print()


# 3. 병렬 처리로 문장 추출 실행
print("병렬 문장 추출 시작...")
result_df = extract_aspect_sentences(df, aspect_terms)
print()

# 4. 결과 저장
result_df.to_csv('/content/drive/MyDrive/fin/0420_사회_경북산불_aspect_sentences.csv', index=False, encoding='utf-8')
print(f"추출된 문장: {len(result_df)}개")

# 5. 결과 샘플 출력
print("\n결과 샘플:")
print(result_df.head())

로드된 aspect terms: 1929개

0        주요 보수 잠룡으로 꼽히는 원희룡 전 국토교통부 장관이 21대 대선 불출마를 선언했...
1        공공기관을 사칭해  물품을 주문한 뒤 잠적하는 사기가 기승을 부리고 있습니다. 지난...
2        역대 최악의 산불로 수많은 피해를 입은 지역들에 최근 수상한 전입신고가 늘어났다. ...
3        국가유산청은 국가유산 재난 위기 경보 수준을 '심각'에서 '경계'로 한 단계 하향했...
4        공공기관을 사칭해  물품을 주문한 뒤 잠적하는 사기가 기승을 부리고 있습니다. 지난...
                               ...                        
18690    3 1절이자 연휴의 시작인 1일 전국에 비가 내린다. 기온은 평년보다 높아 온화할 ...
18691    단일 규모로 국내 최대 산불인 2022년 3월 울진 삼척산불 때 경북 울진군의 금강...
18692    대형산불이 빈발하는 봄철을 앞두고 동해안에 비상이 걸렸다. 예년과 달리 동해안 일대...
18693    3 1절인 1일 전국 대부분 비가 내린다. 중부지방과 경상권을 중심으로 매우 건조할...
18694    3 1절인 1일 토요일은 평년보다 기온이 높을 것으로 보이며 오후부터 전국에 비가 ...
Name: content, Length: 18695, dtype: object
로드된 기사: 18695개

병렬 문장 추출 시작...



추출된 문장: 114052개

결과 샘플:
                                   original_sentence  \
0   주요 보수 잠룡으로 꼽히는 원희룡 전 국토교통부 장관이 21대 대선 불출마를 선언했다.   
1   주요 보수 잠룡으로 꼽히는 원희룡 전 국토교통부 장관이 21대 대선 불출마를 선언했다.   
2  윤석열 전 대통령 파면으로 인해 치러지는 대선인 만큼 친윤(친윤석열)계 정치인으로서...   
3  윤석열 전 대통령 파면으로 인해 치러지는 대선인 만큼 친윤(친윤석열)계 정치인으로서...   
4  원 전 장관은 9일 자신의 사회관계망서비스(SNS)를 통해 "저는 이번 대선에 출마...   

                                  processed_sentence term  
0       주요 보수 잠룡 꼽히는 원희룡 전 국토교통부 장관 21대 대선 불출마 선언했다.   장관  
1       주요 보수 잠룡 꼽히는 원희룡 전 국토교통부 장관 21대 대선 불출마 선언했다.  장관   
2  윤석열 전 대통령 파면 인해 치러지는 대선인 만큼 친윤(친윤석열)계 정치인 책임 지...  대통령  
3  윤석열 전 대통령 파면 인해 치러지는 대선인 만큼 친윤(친윤석열)계 정치인 책임 지...  정치인  
4  원 전 장관 9일 자신 사회관계망서비스(SNS) 통해 "저 이번 대선 출마하지 않는...  SNS  


In [ ]:
term별 문장 수 계산
term_counts = result_df.groupby('term')['target_sentence'].nunique().reset_index()
term_counts.columns = ['term', 'sentence_count']

# 문장 수 기준 내림차순 정렬
term_counts = term_counts.sort_values('sentence_count', ascending=False)

# 결과 저장
term_counts.to_csv('/content/drive/MyDrive/fin/term_sentence_counts.csv', index=False, encoding='utf-8')

# 상위 10개 term 출력
print("상위 10개 term:")
print(term_counts.head())